# Settings

In [1]:
import platform  

def get_device():
    """
    Function to determine the device type based on the node name.
    It uses a dictionary to map node names to device types.

    :return: device type as a string
    :raises Exception: if node name is not found in the device_map dictionary
    """
    device_map = {
        "PC-Cristian": "cuda",
        "Dell-G5-15-Alexios": "cuda",
        "MacBook-Pro-di-Cristian.local": "mps",
        "MacBookProDiGrazia": "cpu",
        "DESKTOP-RQVK8SI":"cuda",
        "MacBook-Pro.station":"mps"
    }

    try:
        return device_map[platform.uname().node]
    except KeyError:
        raise Exception("Node name not found. Please add your node name and its corresponding device to the dictionary.")

device = get_device()

In [2]:
IMAGE_SIZE = 256

In [3]:
import os
from project_paths import *

def delete_ds_store_files(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted {file_path}")

# Specifica la cartella principale in cui cercare i file .DS_Store
folder_path = data_folder_path

delete_ds_store_files(folder_path)

# Create YOLO dataset folder

In [4]:
import os, shutil, random

def create_yolo_datset(splitted_frames_path, splitted_gt_path, yolo_dataset_path, num_frames_per_video=10):

    # folders
    # if yolo dataset folder, doesn't exist, create it
    if not os.path.exists(yolo_dataset_path):
        os.makedirs(yolo_dataset_path)  

    # create the yolo dataset structure, we create a folder for train and a folder for val
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val'))

    # create the two inner folders: one for fire and one for no fire
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train', 'fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train', 'fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train', 'no_fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train', 'no_fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val', 'fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val', 'fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val', 'no_fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val', 'no_fire'))

    # now we copy the images in the right folder
    # we have to copy the images in the train folder and in the val folder
    # for each folder in SPLITTED_FRAMES_DATASET/TRAINING_SET/0 we randomly sample num_frames_per_video frames and put them in dataset/train/no_fire
    # for each folder in SPLITTED_FRAMES_DATASET/VALIDATION_SET/0 we randomly sample num_frames_per_video frames and put them in dataset/val/no_fire

    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        for _ in range (num_frames_per_video):
            # randomly sample a frame
            frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder)))
            while frame in chosen_frames:
                frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder)))
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder, frame), os.path.join(yolo_dataset_path, 'train', 'no_fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'train', 'no_fire', frame), os.path.join(yolo_dataset_path, 'train', 'no_fire', str(count)+'.jpg'))
            count += 1
        
    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        for _ in range (num_frames_per_video):
            # randomly sample a frame
            frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder)))
            while frame in chosen_frames:
                frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder)))
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder, frame), os.path.join(yolo_dataset_path, 'val', 'no_fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'val', 'no_fire', frame), os.path.join(yolo_dataset_path, 'val', 'no_fire', str(count)+'.jpg'))
            count += 1

    # # for each folder in SPLITTED_FRAMES_DATASET/TRAINING_SET/1 we randomly sample num_frames_per_video frames starting from the frame indicated in the annotation file
    # # and put them in dataset/train/fire
    # # for each folder in SPLITTED_FRAMES_DATASET/VALIDATION_SET/1 we randomly sample num_frames_per_video frames starting from the frame indicated in the annotation file
    # # and put them in dataset/val/fire

    count = 0    
    num_frames = 0
    num_folders = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1')):
        num_folders += 1
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        # get the annotation file corresponding to the folder
        annotation_file = folder.replace("mp4", "rtf")
        # open the annotation file
        with open(os.path.join(splitted_gt_path, 'TRAINING_SET', '1', annotation_file), 'r') as f:
            # get the frame in which the fire starts
            start_frame = int(f.readline().split(',')[0])
            # randomly sample a frame whose number starts from start_frame
            available_frames = []
            for _ in range (num_frames_per_video):
                for frame in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1', folder)):
                    if int(frame.split('.')[0])>=start_frame:
                        available_frames.append(frame)
                frame = random.choice(available_frames)
                while frame in chosen_frames:
                    frame = random.choice(available_frames)
                chosen_frames.append(frame)
                num_frames += 1
                # copy the frame in the right folder
                shutil.copy(os.path.join(splitted_frames_path, 'TRAINING_SET','1', folder, frame), os.path.join(yolo_dataset_path, 'train', 'fire', frame))
                # rename the frame
                os.rename(os.path.join(yolo_dataset_path, 'train', 'fire', frame), os.path.join(yolo_dataset_path, 'train', 'fire', str(count)+'.jpg'))
                count += 1

    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        # get the annotation file corresponding to the folder
        annotation_file = folder.replace("mp4", "rtf")
        # open the annotation file
        with open(os.path.join(splitted_gt_path, 'VALIDATION_SET', '1', annotation_file), 'r') as f:
            # get the frame in which the fire starts
            start_frame = int(f.readline().split(',')[0])
            # randomly sample a frame whose number starts from start_frame
            available_frames = []
            for _ in range (num_frames_per_video):
                for frame in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1', folder)):
                    if int(frame.split('.')[0])>=start_frame:
                        available_frames.append(frame)
                frame = random.choice(available_frames)
                while frame in chosen_frames:
                    frame = random.choice(available_frames)
                chosen_frames.append(frame)
                # copy the frame in the right folder
                shutil.copy(os.path.join(splitted_frames_path, 'VALIDATION_SET','1', folder, frame), os.path.join(yolo_dataset_path, 'val', 'fire', frame))
                # rename the frame
                os.rename(os.path.join(yolo_dataset_path, 'val', 'fire', frame), os.path.join(yolo_dataset_path, 'val', 'fire', str(count)+'.jpg'))
                count += 1

create_yolo_datset(splitted_frames_path, splitted_annotations_path, yolo_folder_path, num_frames_per_video=5)


# Preprocess images

In [5]:
# open all the images in yolo_dataset and resize them to 224x224
from PIL import Image
from tqdm import tqdm
import os

def resize_images(dataset_path, img_size=224):

    folders = ['train', 'val']
    subfolders = ['fire', 'no_fire']

    for folder in folders:
        for subfolder in subfolders:
            for image in tqdm(os.listdir(os.path.join(dataset_path, folder, subfolder))):
                img = Image.open(os.path.join(dataset_path, folder, subfolder, image))
                img = img.resize((img_size, img_size))
                img.save(os.path.join(dataset_path, folder, subfolder, image))

resize_images(yolo_folder_path, IMAGE_SIZE)

100%|██████████| 14/14 [00:00<00:00, 199.70it/s]


# Train YOLO

In [6]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m-cls.pt")  # load a pretained model

model.train(data=yolo_folder_path, imgsz=IMAGE_SIZE, epochs=50, batch = 128, device=device, save_period=1)  # train the model

100%|██████████| 32.7M/32.7M [00:19<00:00, 1.72MB/s]
Ultralytics YOLOv8.0.119 🚀 Python-3.10.10 torch-2.0.0 MPS
yolo/engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/Users/cristian/Desktop/MachineLearning/yolo_dataset, epochs=50, patience=50, batch=128, imgsz=256, save=True, save_period=1, cache=False, device=mps, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchsc

# Test YOLO

In [ ]:
import os
from ultralytics import YOLO

In [ ]:

# Evaluate a trained model
yolo_weights_path = "../weights/yolo.pt"
test_videos_folder = "../test_data/TEST_SET"
test_frames_folder = "../yolo_dataset/test"

# create the test frames folder if it doesn't exist
if not os.path.exists(test_frames_folder):
    os.mkdir(test_frames_folder)

model = YOLO(yolo_weights_path)  # load a trained model

In [ ]:
class Profile(contextlib.ContextDecorator):
    """
    Profile class for profiling execution time. 
    Can be used as a decorator with @Profile() or as a context manager with 'with Profile():'.

    Attributes
    ----------
    t : float
        Accumulated time.
    cuda : bool
        Indicates whether CUDA is available.

    Methods
    -------
    __enter__()
        Starts timing.
    __exit__(type, value, traceback)
        Stops timing and updates accumulated time.
    time()
        Returns the current time, synchronizing with CUDA if available.
    """
    
    def __init__(self, t=0.0):
        """
        Initializes the Profile class.

        Parameters:
        t : float
            Initial accumulated time. Defaults to 0.0.
        """
        self.t = t  # Accumulated time
        self.cuda = torch.cuda.is_available()  # Checks if CUDA is available

    def __enter__(self):
        """
        Starts timing.
        
        Returns:
        self
        """
        self.start = self.time()  # Start time
        return self

    def __exit__(self, type, value, traceback):
        """
        Stops timing and updates accumulated time.
        
        Parameters:
        type, value, traceback : 
            Standard parameters for an exit method in a context manager.
        """
        self.dt = self.time() - self.start  # Delta-time
        self.t += self.dt  # Accumulates delta-time

    def time(self):
        """
        Returns the current time, synchronizing with CUDA if available.
        
        Returns:
        float
            The current time.
        """
        if self.cuda:  # If CUDA is available
            torch.cuda.synchronize()  # Synchronizes with CUDA
        return time.time()  # Returns current time

In [ ]:
import cv2, os, glob, tqdm, torch
import pandas as pd

# create dataframe to which save if a clip is fire or not and the discovering frame
df = pd.DataFrame(columns=['video', 'fire', 'frame'])
processed_frames = 0
computation_time = 0.0
dt = Profile()
if torch.cuda.is_available():
        memory = []

for video in tqdm.tqdm(glob.glob(os.path.join(test_videos_folder, "**"), recursive=True)):
        
    if os.path.isdir(video):
        continue
    # Process the video
    ret = True
    cap = cv2.VideoCapture(video) # Decodifica lo streaming
    fps = cap.get(cv2.CAP_PROP_FPS) # Ottiene il frame rate
    f = 0
    fire = 0
    while ret:
        ret, img = cap.read() # Chiamando read leggiamo il frame successivo dallo stream
        if ret: # ret è false quando non ci sono più frame da leggere
            f += 1
            # Il tensore img letto viene trasformato tramite la classe PIL e lo salviamo
            frame_name = os.path.join(test_frames_folder, "{:05d}.jpg".format(f))
            
            cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
            cv2.imwrite(frame_name, img)
            # evaluate YOLO model on the frame
            with dt:
                results = model.predict(frame_name, verbose=False)
            computation_time += dt.dt
            processed_frames += 1
            # if it is a fire tell the video is fire and continue with the next video
            for result in results:
                if result.probs.data.tolist()[0] >= 0.5:
                    fire = 1
                    ret = 0
                    break
    # save the result in the dataframe
    df.loc[len(df)] = [video, fire, round(f/fps)]
    #empty the test frames folder
    # for frame in os.listdir(test_frames_folder):
    #     os.remove(os.path.join(test_frames_folder, frame))

    # Calcolo delle dimensioni della memoria occupata
    if torch.cuda.is_available():
        memory_bytes = torch.cuda.memory_allocated()
        memory.append( memory_bytes / (1024 ** 2))
    
    cap.release()

In [ ]:
annotations_path = '../test_data/GT_TEST_SET'
# modify the first column of the dataframe to remove the path and keep only the video name
df['video'] = df['video'].apply(lambda x: x.split('/')[-1])

In [ ]:
from sklearn.metrics import precision_score, recall_score

tp = 0
fp = 0
fn = 0
tn = 0
delays = []

video_counter = 0
GUARD_TIME = 5 # seconds
MEM_TARGET = 4000 # MB
PFR_TARGET = 10 

for i in range(len(df)):

    annotation_file = df['video'][i].replace("mp4", "rtf")

    with open(os.path.join(annotations_path, annotation_file), 'r') as f:

        line = f.readline()

        if line and df['fire'][i] == 1:
            # There is a fire in the video
            g_frame = int(line.split(',')[0])
            p_frame = df['frame'][i]
            
            if p_frame >= max(0, g_frame - GUARD_TIME):
                # Detection is fast enough
                delays.append(abs(p_frame - g_frame))
                tp += 1
            else:
                # Detection is not fast enough
                fp += 1
        elif not line and df['fire'][i]==1:
            fp += 1
        elif line and df['fire'][i]==0:
            fn += 1
        elif not line and df['fire'][i]==0:
            tn += 1
        else:
            print("Something went wrong")

# Compute precision, recall and f1 score
# Count the number of true positives, false positives and false negatives


try:
    precision = tp/(tp+fp)
except ZeroDivisionError:
    precision = 0
try:
    recall = tp/(tp+fn)
except ZeroDivisionError:
    recall = 0

try:
    D = sum(delays)/len(delays) 
    Dn = max(0, 60-D)/60
except:
    print("Can't calculate D because no fire detected in the test set")
    D = float("inf")
    Dn = 0

f_score = 2 * precision * recall / (1e-10 + precision + recall)
pfr = 1 /(computation_time / processed_frames)
mem = memory_per_video_occupancy.mean().item()

pfr_delta = max(0, PFR_TARGET/pfr - 1)
mem_delta = max(0, mem/MEM_TARGET - 1)
fds = (precision * recall * Dn) / ((1 + pfr_delta) * (1 + mem_delta))
accuracy = (tp + tn) / (tp + tn + fp + fn)

# Print results
print("..:: RESULTS ::..")

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F-score: {:.4f}".format(f_score))
print("Average notification delay: {:.4f}".format(D))
print("Normalized average detection delay: {:.4f}".format(Dn))
print("Processing frame rate: {:.4f}".format(pfr))
print("Memory usage: {:.4f}".format(mem))
print("Final detection score: {:.4f}".format(fds))